### Will a Customer Accept the Coupon?

**Context**

Imagine driving through town and a coupon is delivered to your cell phone for a restaraunt near where you are driving. Would you accept that coupon and take a short detour to the restaraunt? Would you accept the coupon but use it on a sunbsequent trip? Would you ignore the coupon entirely? What if the coupon was for a bar instead of a restaraunt? What about a coffee house? Would you accept a bar coupon with a minor passenger in the car? What about if it was just you and your partner in the car? Would weather impact the rate of acceptance? What about the time of day?

Obviously, proximity to the business is a factor on whether the coupon is delivered to the driver or not, but what are the factors that determine whether a driver accepts the coupon once it is delivered to them? How would you determine whether a driver is likely to accept a coupon?

**Overview**

The goal of this project is to use what you know about visualizations and probability distributions to distinguish between customers who accepted a driving coupon versus those that did not.

**Data**

This data comes to us from the UCI Machine Learning repository and was collected via a survey on Amazon Mechanical Turk. The survey describes different driving scenarios including the destination, current time, weather, passenger, etc., and then ask the person whether he will accept the coupon if he is the driver. Answers that the user will drive there ‘right away’ or ‘later before the coupon expires’ are labeled as ‘Y = 1’ and answers ‘no, I do not want the coupon’ are labeled as ‘Y = 0’.  There are five different types of coupons -- less expensive restaurants (under \\$20), coffee houses, carry out & take away, bar, and more expensive restaurants (\\$20 - \\$50). 

**Deliverables**

Your final product should be a brief report that highlights the differences between customers who did and did not accept the coupons.  To explore the data you will utilize your knowledge of plotting, statistical summaries, and visualization using Python. You will publish your findings in a public facing github repository as your first portfolio piece. 





### Data Description
Keep in mind that these values mentioned below are average values.

The attributes of this data set include:
1. User attributes
    -  Gender: male, female
    -  Age: below 21, 21 to 25, 26 to 30, etc.
    -  Marital Status: single, married partner, unmarried partner, or widowed
    -  Number of children: 0, 1, or more than 1
    -  Education: high school, bachelors degree, associates degree, or graduate degree
    -  Occupation: architecture & engineering, business & financial, etc.
    -  Annual income: less than \\$12500, \\$12500 - \\$24999, \\$25000 - \\$37499, etc.
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she buys takeaway food: 0, less than 1, 1 to 3, 4 to 8 or greater
    than 8
    -  Number of times that he/she goes to a coffee house: 0, less than 1, 1 to 3, 4 to 8 or
    greater than 8
    -  Number of times that he/she eats at a restaurant with average expense less than \\$20 per
    person: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    

2. Contextual attributes
    - Driving destination: home, work, or no urgent destination
    - Location of user, coupon and destination: we provide a map to show the geographical
    location of the user, destination, and the venue, and we mark the distance between each
    two places with time of driving. The user can see whether the venue is in the same
    direction as the destination.
    - Weather: sunny, rainy, or snowy
    - Temperature: 30F, 55F, or 80F
    - Time: 10AM, 2PM, or 6PM
    - Passenger: alone, partner, kid(s), or friend(s)


3. Coupon attributes
    - time before it expires: 2 hours or one day

In [53]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.express as px

### Problems

Use the prompts below to get started with your data analysis.  

1. Read in the `coupons.csv` file.




In [54]:
data = pd.read_csv('data/coupons.csv')

In [55]:
data.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


In [56]:
# number of rows that have na fields

print('NA Rows: ', data.isnull().any(axis=1).sum())
print('Total Rows: ', data.size)
print('Rows with NA (%): ', round((data.isnull().any(axis=1).sum() / data.size) * 100, 2))

NA Rows:  12576
Total Rows:  329784
Rows with NA (%):  3.81


In [57]:
# get the number of recurrances for the data type
print(data.dtypes.value_counts())

# extract the ordinal_columns (numeric) and nominal_columns (non-columns)
ordinal_columns = data.select_dtypes("int64").columns
nominal_columns = data.select_dtypes("object").columns

print('ordinal columns', ordinal_columns)
print('nomial_columns', nominal_columns)

object    18
int64      8
Name: count, dtype: int64
ordinal columns Index(['temperature', 'has_children', 'toCoupon_GEQ5min', 'toCoupon_GEQ15min',
       'toCoupon_GEQ25min', 'direction_same', 'direction_opp', 'Y'],
      dtype='object')
nomial_columns Index(['destination', 'passanger', 'weather', 'time', 'coupon', 'expiration',
       'gender', 'age', 'maritalStatus', 'education', 'occupation', 'income',
       'car', 'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20',
       'Restaurant20To50'],
      dtype='object')


In [58]:
# let's check if all the ordinal columns have values
count_null_values_in_ordinal_col = data[ordinal_columns].isna().sum()
count_null_values_in_nominal_Cols = data[nominal_columns].isna().sum()

print(count_null_values_in_ordinal_col, count_null_values_in_nominal_Cols)

temperature          0
has_children         0
toCoupon_GEQ5min     0
toCoupon_GEQ15min    0
toCoupon_GEQ25min    0
direction_same       0
direction_opp        0
Y                    0
dtype: int64 destination                 0
passanger                   0
weather                     0
time                        0
coupon                      0
expiration                  0
gender                      0
age                         0
maritalStatus               0
education                   0
occupation                  0
income                      0
car                     12576
Bar                       107
CoffeeHouse               217
CarryAway                 151
RestaurantLessThan20      130
Restaurant20To50          189
dtype: int64


In [59]:
nominal_value_analysis = {"col_name": [], "values": [], "na_ratio": [], "no_of_values": []}
total_rows = data.size

for col in nominal_columns:
    unique_v = data[col].unique()
    unique_size = unique_v.size
    nominal_value_analysis['no_of_values'].append(unique_size)
    nominal_value_analysis['col_name'].append(col)
    nominal_value_analysis['values'].append(unique_v)
    nominal_value_analysis['na_ratio'].append(unique_size/total_rows)

nominal_value_analysis = pd.DataFrame(nominal_value_analysis)
print(nominal_value_analysis)


                col_name                                             values  \
0            destination                      [No Urgent Place, Home, Work]   
1              passanger                [Alone, Friend(s), Kid(s), Partner]   
2                weather                              [Sunny, Rainy, Snowy]   
3                   time                        [2PM, 10AM, 6PM, 7AM, 10PM]   
4                 coupon  [Restaurant(<20), Coffee House, Carry out & Ta...   
5             expiration                                           [1d, 2h]   
6                 gender                                     [Female, Male]   
7                    age          [21, 46, 26, 31, 41, 50plus, 36, below21]   
8          maritalStatus  [Unmarried partner, Single, Married partner, D...   
9              education  [Some college - no degree, Bachelors degree, A...   
10            occupation  [Unemployed, Architecture & Engineering, Stude...   
11                income  [$37500 - $49999, $62500 -

In [60]:
data.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


2. Investigate the dataset for missing or problematic data.

In [61]:
data.isna()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12680,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12681,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12682,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


3. Decide what to do about your missing data -- drop, replace, other...

In [62]:
def value_count(columns, file_name=None):
    if isinstance(columns, str):
        columns = [columns]
    for col_name in columns:
        v = data[col_name].value_counts().reset_index().rename(columns={"index": "value", 0: "count"})
        fig = px.pie(v, values="count",
                    title=f"Feature = '{col_name}' value counts",
                    names=col_name, hole=.4)
        if file_name:
            fig.write_image(f'./images/{file_name}.jpg')
        else:
            fig.write_image(f'./images/vcount_{col_name}.jpg')
        fig.show()

value_count(nominal_columns)

In [63]:
c = ['Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']
# fig = px.histogram(data, x=[["Restaurant20To50"]])
# fig.show()

new_Frame=data[c].value_counts().reset_index().rename(columns={"index": "value", 0: "count"})
collect_data = {}
for i in c:
    for t in data[i].unique():
        if not isinstance(t, str):
            continue
        collect_data[t] = {}

# print(new_Frame)
def pivot_on_col(column_name, collect_data):
    p=new_Frame.pivot_table(columns=column_name, values='count', aggfunc='sum')
    for el in p.columns:
        collect_data[el].update({column_name: {}})

    pv_frame: dict = pd.DataFrame(p).to_dict()

    for r, v in pv_frame.items():
        collect_data[r].update({column_name: v['count']})
    return collect_data


for i in c:
    collect_data = pivot_on_col(i, collect_data)

tranformed = pd.DataFrame(collect_data).reset_index().set_index('index')#.rename(columns={"index": "feature"}).set_index('feature')
tranformed.dropna()
print(tranformed)
fig = px.bar(tranformed, labels="index", title='Coupon Utilization Across Outlets')
fig.write_image('./images/utilization_across.jpg')
fig.show()


                      never  less1   1~3   gt8   4~8
index                                               
Bar                    4934   3394  2370   327  1054
CoffeeHouse            2852   3298  3159  1111  1659
CarryAway               131   1790  4502  1573  4083
RestaurantLessThan20    176   2005  5163  1285  3450
Restaurant20To50       2070   5859  3202   264   684


In [64]:
data['occupation'] = data['occupation'].apply(lambda x: x if x in ['Unemployed', 'Student', 'Retired'] else 'Employed')

value_count('occupation', 'occupation_modified')

4. What proportion of the total observations chose to accept the coupon? 



In [65]:
round(data['Y'].value_counts()[1.0]/data['Y'].count() * 100, 2)

56.84

5. Use a bar plot to visualize the `coupon` column.

In [66]:
fig = px.bar(data['coupon'].value_counts(), labels='coupon', title='Coupons Distribution')
fig.write_image('./images/5_coupon_wiz.jpg')
fig.show()


6. Use a histogram to visualize the temperature column.

In [67]:
fig = px.histogram(data, x='temperature',
                   labels='temperature',
                    title='Temperature Distribution')
fig.write_image('./images/6_temp_wiz.jpg')
fig.update_layout(bargap=0.2)
fig.show()

print(data['temperature'].value_counts())

temperature
80    6528
55    3840
30    2316
Name: count, dtype: int64


**Investigating the Bar Coupons**

Now, we will lead you through an exploration of just the bar related coupons.  

1. Create a new `DataFrame` that contains just the bar coupons.


In [68]:
bar_df = data[data['coupon']=='Bar']


2. What proportion of bar coupons were accepted?


In [69]:
accepted_bar_df = bar_df[bar_df['Y']==1]
round(100*accepted_bar_df.shape[0]/data.shape[0], 2)

6.52

3. Compare the acceptance rate between those who went to a bar 3 or fewer times a month to those who went more.


In [70]:
tdf = accepted_bar_df['Bar'].value_counts()

three_or_fewer = tdf['1~3'] #+ tdf['less1']
more_times = tdf.sum() - three_or_fewer - tdf['never']

print(three_or_fewer)
print(more_times)
print('ratio: ', round(100 * (three_or_fewer / more_times) , 2))


257
406
ratio:  63.3


4. Compare the acceptance rate between drivers who go to a bar more than once a month and are over the age of 25 to the all others.  Is there a difference?


In [95]:

ages_above_25 = ['26', '31', '50plus', '36', '41', '46']
df_25_above = accepted_bar_df.query('age in @ages_above_25 and Bar  not in ["never"]')
df_25_above['Bar'].value_counts()

tdf = df_25_above.query('Bar not in ["never"]')
tdf['Bar'].value_counts()

driver_who_go_bar_more_than_once = df_25_above.query('Bar not in ["less1"]').shape[0]
# print(driver_who_go_bar_more_than_once)


df_25_less = accepted_bar_df.query('age not in @ages_above_25 and Bar  not in ["never"]')
less_than_25_bar_goers = df_25_less['Bar'].shape[0]

print(round(100*driver_who_go_bar_more_than_once / less_than_25_bar_goers, 2))

print(round(100*driver_who_go_bar_more_than_once / tdf.shape[0], 2))

print(round(100*driver_who_go_bar_more_than_once / accepted_bar_df.shape[0], 2))

159.57
62.11
36.28


5. Use the same process to compare the acceptance rate between drivers who go to bars more than once a month and had passengers that were not a kid and had occupations other than farming, fishing, or forestry. 


In [91]:
bar_more_than_once = ['less1', 'never']
driver_bar_more_than_ones = accepted_bar_df.query('Bar not in @bar_more_than_once')
count_driver_bar_more_than_ones = driver_bar_more_than_ones.shape[0]


all_non_kids_forstry_occ = data.query('passanger not in ["Kid(s)"] and occupation not in ["Farming Fishing & Forestry"]')
count_all_non_kids_forstry_occ = all_non_kids_forstry_occ.shape[0]

print(round(100*count_driver_bar_more_than_ones/count_all_non_kids_forstry_occ,2))

3.58


6. Compare the acceptance rates between those drivers who:

- go to bars more than once a month, had passengers that were not a kid, and were not widowed *OR*
- go to bars more than once a month and are under the age of 30 *OR*
- go to cheap restaurants more than 4 times a month and income is less than 50K. 



In [100]:
month_tags = ['4-8', 'gt8']
month_bar = accepted_bar_df.query('Bar in @month_tags')
pass_no_kid_not_widow = month_bar.query('passanger != "Kid(s)" and maritalStatus not in ["Widowed"]')
count_pass_no_kid_not_widow = pass_no_kid_not_widow.shape[0]

print('#  Pass not a kid and not widowed', count_pass_no_kid_not_widow)

under_30_tags = ['21', '26', 'below21']
month_more_than_once = ['1~3', '4~8', 'g8']
df2 = accepted_bar_df.query('Bar in @month_more_than_once and age in @under_30_tags')
count_under_30_more_than_once = df2.shape[0]
print('# Under 30 and more than once', count_under_30_more_than_once)


#  Pass not a kid and not widowed 35
# Under 30 and more than once 219


7.  Based on these observations, what do you hypothesize about drivers who accepted the bar coupons?

### Independent Investigation

Using the bar coupon example as motivation, you are to explore one of the other coupon groups and try to determine the characteristics of passengers who accept the coupons.  